# Shap Seed Reruns

https://lime-ml.readthedocs.io/en/latest/lime.html

random_state – an integer or numpy.RandomState that will be used to generate random numbers. If None, the random state will be initialized using the internal numpy seed.

https://shap.readthedocs.io/en/latest/generated/shap.Explainer.html

seed: None or int
seed for reproducibility



Initial runs had the following to reset seed
```
def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True
```


SHAP may not have had adjusted seed values, so will rerun with seeds